In [2]:
from termcolor import colored
import pandas as pd
import numpy as np
import string
import regex as re
import zipfile
import gc
from scipy.stats import boxcox
import sys 
# other python utilities 
from collections import Counter 
from tqdm import tqdm 
import pandas as pd 
import numpy as np 
import warnings, math
from termcolor import colored
import pickle
import string

# for eval
from sklearn.model_selection import train_test_split
import random

# for SVD
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix
pd.set_option('display.float_format','{:.2f}'.format)

# for TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy

In [3]:
df = pd.read_csv("D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\Oct_Forth_projectType.csv")

categories_cols = []

for col in df.columns:
    #print(str(col))
    cats = len(df[col].unique())
    #print(cats)
    if cats<=10:
        categories_cols.append(col)
        print(col)
        
df[categories_cols] = df[categories_cols].astype('category')

Project Grade Level Category
Project Resource Category
Project Current Status
School Metro Type
School State
Donor State


In [4]:
df.head(1)

,Project ID,Donation ID,Donor ID,Donation Amount,Donation Received Date,School ID,Project Title,Project Need Statement,Project Subject Category Tree,Project Subject Subcategory Tree,...,Project Cost,Project Posted Date,Project Expiration Date,Project Current Status,Project Fully Funded Date,School Metro Type,School State,School Zip,Donor State,Donor Zip
0,0029e426fd3296af4fc333580fa895fe,061a5652846763d2eca84535978a647b,0e345dcdef0d2a36c9bd17bf1ac3e10a,0.68,2014-08-10 20:32:12,55fe7ff4c71893998d0e9a26a9d3d0b7,"Everyone Needs an Address, Especially Maniac M...",My students need a class set of the book Mania...,"Applied Learning, Literacy & Language","Character Education, Literature & Writing",...,510.00,2014-06-12,2014-10-09,Fully Funded,2014-08-23,suburban,Georgia,30238,California,900.00


In [5]:
# for only having the preference / donation amount / event strength
donor_project = df.groupby(['Donor ID', 'Project ID'])['Donation Amount'].sum().reset_index()
print(f"{len(donor_project)}")
print(f"{len(df)}")


22919
48546


In [6]:
"""# create a df for 

donors_projects_df = donor_project.pivot('Donor ID', 'Project ID', 'Donation Amount').fillna(0)

donors_projects = donors_projects_df.values
donors_id = donors_projects_df.index
projects_id = donors_projects_df.columns

print(f'length of donors: {len(donors_id)}')
print(f'length of projects: {len(projects_id)}')
"""

In [8]:

# create a df for projects specifications
projects_df = df[['Project ID', 'Project Subject Subcategory Tree', 'Project Title', 'Project Need Statement', 'School State']].drop_duplicates().reset_index(drop=True)
projects_id = df['Project ID'].tolist()

# clean the text
patterns = ['Â', '']
punct = re.sub(r'[\&\.\,]', '', string.punctuation)

def clean(text):
    for pattern in patterns:
        text = re.sub(pattern, ' ', text)
    text = text.translate(str.maketrans('', '', punct))
    text = text.lower()
    text = re.sub(r' +(?![iaIA])[\w] +', ' ', text)
    text = re.sub(r' \s+', ' ', text)
    
    return text

# add them up
project_txt = []

for i in tqdm(range(len(projects_df)), position=0, leave=True):
    project_txt.append(
        clean(projects_df.loc[i, 'Project Title'] + ' & ' + projects_df.loc[i, 'Project Need Statement'] + projects_df.loc[i, 'Project Subject Subcategory Tree'] + ' & ' + projects_df.loc[i, 'School State'])
        )

projects_df.loc[:, 'Project TFIDF Text'] = project_txt

100%|██████████| 860/860 [00:00<00:00, 14136.35it/s]


In [20]:
# TFIDF
vectorizer = TfidfVectorizer(max_df = 0.99,    
                             min_df = 0.001, # better change to 0.9 for less memory usage
                             stop_words='english',
                             strip_accents='unicode',
                             analyzer='word')

tfidf_matrix = vectorizer.fit_transform(project_txt)

print(f"tfidf_matrix.shape = {tfidf_matrix.shape}")

tfidf_matrix.shape = (860, 3146)


In [ ]:

# =======================TFIDF=======================#

# -------------------projects-------------------#

# single project 
def get_project_profile_tfidf(project_id, tfidf_matrix):
    # get the ids
    idx = projects_id.index(project_id)
    # get the word count vector for that document
    project_profile = tfidf_matrix[idx:idx+1]
    return project_profile

# multiple projects
# np.ravel creates a flattened 1d array
def get_projects_profiles_tfidf(ids, tfidf_matrix):
    # repeat for all projects
    profiles_list = [get_project_profile_tfidf(project_id, tfidf_matrix) for project_id in np.ravel([ids])]
    # stack them onto each other
    project_profiles = scipy.sparse.vstack(profiles_list)

    return project_profiles


# --------------donors------------------#
def build_donor_profile_tfidf(donor_id, df_train_donor_index, tfidf_matrix=tfidf_matrix):

    # get the id of each person and the projects they
    # donated to
    donor_train_donations_serie = df_train_donor_index.loc[donor_id]
    # print(f'len of donations in train: {len(donor_train_donations_serie)}')


    # get the vectors of projects this person has donated to
    donor_project_profiles = get_projects_profiles_tfidf(donor_train_donations_serie['Project ID'], tfidf_matrix)
    # get the smoothed donated amount as the weight of each project
    donor_project_strengths = np.array(donor_train_donations_serie['Donation Amount']).reshape(-1, 1)

    # multiply the weights and vectors
    multiplication = donor_project_profiles.multiply(donor_project_strengths)
    # now we normalize the whole vector 
    normalized_donor_preference = preprocessing.normalize(np.sum(multiplication, axis=0))


    return normalized_donor_preference



def build_donors_profiles_tfidf(df_test_donor_index, df_train_donor_index, sample_size = 1000):
    
    # now for all donors we build a profile in a dictionary
    donor_profiles = {}
    donors_in_test_set = df_test_donor_index.index.unique().tolist()[:sample_size]

    for donor_id in tqdm(donors_in_test_set[:sample_size], position=0, leave=True):
        donor_profiles[donor_id] = build_donor_profile_tfidf(donor_id, df_train_donor_index)

    return donor_profiles

